# Dimension reduction by lasso and backwards elimination

This is a script to get fewer variables by lasso from high dimensional data, and then apply backwards elimination to build linear regression model with small p-value and small number of predictors.

# Package

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statistics
import math
import statsmodels.api as sm

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers

# Data - train and test split

In [ ]:
X_train_df
y_train_df
X_test_df

# Hyperparameter tuning with GridSearchCV

In [ ]:
k = 10
alpha_lower = 0.01
alpha_upper = 0.1

# Setup the parameter grid
alpha_space = np.arange(alpha_lower, alpha_upper, 0.01)
param_grid = {'alpha':alpha_space}

# Instantiate a lasso regression
lasso = Lasso(normalize = True)

# Instantiate the GridSearchCV object
lasso_cv = GridSearchCV(lasso, param_grid, cv = k)

# Fit it to data
lasso_cv.fit(X_train_df.values, y_train_df.values)

# Calculate training accuracy by RMSE
y_pred = lasso_cv.predict(X_train_df.values)
rmse = math.sqrt(statistics.mean((y_train_df.values - y_pred)**2))

# Predictors
best_alpha = lasso_cv.best_params_['alpha']
lasso = Lasso(alpha = best_alpha, normalize = True)
lasso.fit(X_train_df.values, y_train_df.values)
lasso_coef = lasso.coef_
p = sum(abs(lasso_coef) > 0)

# Print the tuned parameters and score
print("Tuned lasso regression hyperparameters: {}".format(lasso_cv.best_params_))
print("Best score: {0:.2f}".format(lasso_cv.best_score_))
print("RMSE: {0:.2f}".format(rmse))
print("Number of predictors: {}".format(p))

# Correlation among predictors

In [ ]:
COLUMNS = X_train_df.columns
var = pd.Series(COLUMNS[abs(lasso_coef) > 0])
coef = pd.Series(lasso_coef[abs(lasso_coef) > 0])
lasso_result = pd.concat(objs = [var, coef],
                         axis = 1,
                         keys = ['Variable', 'Lasso_coefficient'])
print(lasso_result.iloc[0:5,:])

In [ ]:
COLUMNS = lasso_result['Variable'].tolist()
corrmat = X_train_df[COLUMNS].corr()
corrmat = corrmat.rename_axis(None).rename_axis(None, axis = 1)
corrmat = corrmat.stack().reset_index()
corrmat.columns = ['var_1', 'var_2', 'correlation']
corrmat = corrmat[corrmat['correlation'] != 1]
corrmat.sort_values(by = 'correlation', ascending = False).head(5)

# Lasso predictors p-values

In [ ]:
COLUMNS = lasso_result['Variable'].tolist()
X = X_train_df[COLUMNS].values
X = sm.add_constant(X)
y = y_train_df.values
result = sm.OLS(y, X).fit()
print(result.summary())

# Backwards elimination with p-value control

In [ ]:
# initial values
COLUMNS = np.array(lasso_result['Variable'])
X_train = X_train_df # we don't use validation set to check accuracy
y_train = y_train_df
p_max = 1.0
i = 0 # counter

# store validation result
n = len(COLUMNS)
p_list = np.zeros(n)
max_pval_list = np.zeros(n)

# threshold of p-value
p_threshold = 0.00001

while p_max > p_threshold:
       
    # run OLS regression
    X = X_train[COLUMNS].values
    X = sm.add_constant(X)
    y = y_train.values
    result = sm.OLS(y, X).fit()
    
    # store number of predictors
    p_list[i] = len(COLUMNS)
    
    # extract ols results
    result_df = results_summary_to_dataframe(result)

    # Adding Intercept label
    COLUMNS_int = COLUMNS.copy()
    COLUMNS_int = np.append('Intercept', COLUMNS_int)
    result_df['predictors'] = COLUMNS_int
    
    # check max p-value
    result_nonint = result_df.copy()
    result_nonint = result_nonint.drop(0, axis = 0) # delete intercept row
    max_pval_list[i] = max(result_nonint['pvals']) # store max p-value
    p_max = max(result_nonint['pvals'])
    
    # delete one predictor
    idx_del = result_nonint['pvals'].idxmax()
    result_nonint = result_nonint.drop(idx_del, axis = 0)

    # store list of predictors after drop one predictors
    COLUMNS = np.array(result_nonint['predictors'])
    
    # counter plus one
    i = i + 1

In [ ]:
np.round(p_max, decimals = 5)

In [ ]:
p_list

In [ ]:
np.round(max_pval_list, decimals = 3)

In [ ]:
np.round(result_df, decimals = 4)

In [ ]:
_ = plt.plot(p_list, max_pval_list)
_ = plt.xlabel('Number of predictors')
_ = plt.ylabel('Max p-values')
plt.show()

In [ ]:
result_df['predictors']

# Prediction of training set

In [ ]:
COLUMNS_final = np.array(result_df['predictors'].drop(0, axis = 0))
COLUMNS_final

In [ ]:
X = X_train_df[COLUMNS_final].values
X = sm.add_constant(X)
y = y_train_df.values
model = sm.OLS(y, X).fit()
y_train_pred = model.predict(X)

# Calculate accuracy
rmse = math.sqrt(statistics.mean((y - y_train_pred)**2))
print("RMSE of training: {:5.2f}".format(rmse))

# Prediction of test set

In [ ]:
X_test = X_test_df[COLUMNS_final].values
X_test = sm.add_constant(X_test)
test_pred = model.predict(X_test).round(1) # DREAM allows only 1 decimal point
result = pd.concat([pd.Series(ID_test_df.values), pd.Series(test_pred)],
                  axis = 1,
                  keys = ['SampleID', 'GA'])
result.head()

In [ ]:
result.describe()